# Career Coach Agent: LangGraph Multi-Tool Agentic System

This notebook demonstrates the construction of an advanced agentic system using LangGraph, a framework for building stateful, multi-actor applications with LLMs. The Career Coach Agent exemplifies how multiple specialized tools can be orchestrated to solve complex, multi-step career planning tasks.

## Architecture Overview

The agent implements a graph-based workflow where:
- **State**: Maintains conversation history and intermediate results
- **Nodes**: Represent computational steps (LLM reasoning, tool execution)
- **Edges**: Define transitions between nodes (conditional routing based on agent decisions)
- **Tools**: External functions the agent can invoke (CV analysis, job search, etc.)

## LangGraph Fundamentals

### State Management

LangGraph maintains a typed state object that persists across node executions. For conversational agents, the state typically includes:
- **messages**: List of conversation turns (user inputs, agent responses, tool outputs)
- **context**: Additional metadata or intermediate results

State updates are functional: each node returns a dictionary of updates that are merged into the current state.

### Graph Structure

The graph consists of:
1. **Agent Node**: LLM that decides which tool to call or how to respond
2. **Tool Node**: Executes the selected tool and returns results
3. **Conditional Edges**: Route flow based on agent decisions (continue with tools or finish)

This architecture enables the agent to:
- Chain multiple tool calls
- Reason about tool outputs
- Dynamically adjust its strategy

In [5]:
import os
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] =
# Define the state schema
class AgentState(TypedDict):
    """State maintained throughout the agent's execution.

    messages: Conversation history with automatic message accumulation
    """
    messages: Annotated[Sequence[BaseMessage], add_messages]

## Tool Definitions

Each tool is a Python function decorated with `@tool`. The decorator:
- Extracts the function signature for the LLM
- Provides the docstring as the tool description
- Handles serialization of inputs and outputs

The LLM uses tool names and descriptions to decide when to invoke each tool. Well-written docstrings are critical for correct tool selection.

### Tool 1: CV Analysis
Identifies weak spots, missing keywords, and structural issues in the candidate's CV.

In [6]:
@tool
def analyze_cv(cv_text: str, target_role: str) -> str:
    """Analyze a CV to identify weak spots, missing skills, and areas for improvement.

    Args:
        cv_text: The full text of the candidate's CV
        target_role: The role/position the candidate is targeting

    Returns:
        Analysis report with identified weaknesses and recommendations
    """
    # Simulated CV analysis (in production, this could use NLP or LLM-based analysis)
    analysis = f"""CV Analysis for {target_role}:

**Strengths:**
- 3 years of Python development experience
- Experience with backend development

**Weaknesses Identified:**
1. Missing modern Python frameworks (FastAPI, Django)
2. No mention of cloud platforms (AWS, Azure, GCP)
3. Limited testing/CI-CD experience
4. Absence of containerization skills (Docker, Kubernetes)
5. No quantifiable achievements or impact metrics

**Recommendations:**
- Add metrics to demonstrate impact (e.g., "Optimized API reducing response time by 40%")
- Highlight any cloud or DevOps experience
- Include collaborative projects or team leadership
- Mention any open-source contributions
"""
    return analysis


@tool
def research_market_trends(role: str, location: str) -> str:
    """Research current job market trends, in-demand skills, and salary ranges for a specific role and location.

    Args:
        role: The job role to research (e.g., "Python Developer")
        location: Geographic location (e.g., "Paris")

    Returns:
        Market research report with trends and skill demands
    """
    # Simulated market research (in production, this would call real job market APIs)
    report = f"""Market Analysis for {role} in {location}:

**Market Overview:**
- Demand: High (15% YoY growth in job postings)
- Competition: Moderate (avg 45 applications per posting)
- Average Salary: €45,000 - €65,000 (mid-level)

**Top In-Demand Skills (2024):**
1. FastAPI / Django / Flask (85% of postings)
2. Docker & Kubernetes (70%)
3. AWS / Azure / GCP (65%)
4. PostgreSQL / MongoDB (60%)
5. CI/CD pipelines (55%)
6. Unit testing / TDD (50%)
7. Microservices architecture (45%)

**Emerging Trends:**
- Increased demand for LLM integration skills
- Shift toward cloud-native development
- Growing importance of observability/monitoring

**Salary Insights:**
- Entry (0-2 years): €35,000 - €45,000
- Mid (3-5 years): €45,000 - €65,000  ← Your target
- Senior (5+ years): €65,000 - €85,000
"""
    return report


@tool
def find_learning_resources(skills_gap: str) -> str:
    """Find learning resources (courses, tutorials, certifications) to fill identified skill gaps.

    Args:
        skills_gap: Description of the skills the candidate needs to acquire

    Returns:
        Curated list of learning resources with URLs and time estimates
    """
    # Simulated resource finder (in production, this would query course databases)
    resources = f"""Learning Resources for: {skills_gap}

**Recommended Courses:**

1. **FastAPI Complete Course** (Udemy)
   - Duration: 8 hours
   - Cost: €15
   - Focus: Building production-ready APIs

2. **Docker & Kubernetes Mastery** (Coursera)
   - Duration: 4 weeks (3-5 hrs/week)
   - Cost: Free (audit) / €49 (certificate)
   - Provider: Google Cloud

3. **AWS Certified Developer Associate** (A Cloud Guru)
   - Duration: 20 hours
   - Cost: €29/month
   - Includes: Hands-on labs + certification prep

4. **Test-Driven Development with Python** (Free)
   - Platform: Real Python
   - Duration: 6 hours
   - Hands-on exercises included

**Free Resources:**
- FastAPI documentation (official tutorial: 3 hours)
- AWS Free Tier (12 months hands-on practice)
- Docker documentation (get started guide: 2 hours)

**Estimated Timeline:** 6-8 weeks (10 hrs/week) to cover core gaps
"""
    return resources


@tool
def search_jobs(role: str, location: str, salary_min: int) -> str:
    """Search for job opportunities matching the specified criteria.

    Args:
        role: Job title or role
        location: Geographic location
        salary_min: Minimum acceptable salary

    Returns:
        List of matching job opportunities with details
    """
    # Simulated job search (in production, this would query job boards APIs)
    jobs = f"""Top 5 Opportunities for {role} in {location} (€{salary_min}+ salary):

**1. Backend Python Developer - TechCorp Paris**
   - Salary: €50,000 - €58,000
   - Stack: Python, FastAPI, PostgreSQL, Docker, AWS
   - Team: 5-person engineering team
   - Perks: Remote hybrid, RTT, stock options
   - Match Score: 85% (strong technical match)

**2. Python Developer - FinanceAI**
   - Salary: €52,000 - €62,000
   - Stack: Python, Django, Redis, Kubernetes
   - Team: 12-person platform team
   - Perks: Full remote option, lunch vouchers
   - Match Score: 80% (good overall fit)

**3. Software Engineer Python - DataFlow Systems**
   - Salary: €48,000 - €56,000
   - Stack: Python, Flask, MongoDB, GCP
   - Team: 8-person product team
   - Perks: Flexible hours, learning budget €1500/year
   - Match Score: 78% (requires cloud experience)

**4. Python Backend Engineer - GreenTech Solutions**
   - Salary: €53,000 - €60,000
   - Stack: Python, FastAPI, DynamoDB, AWS Lambda
   - Team: 15-person engineering org
   - Perks: Impact-driven mission, equity
   - Match Score: 82% (serverless experience preferred)

**5. Full Stack Python Developer - MediaPlatform**
   - Salary: €51,000 - €59,000
   - Stack: Python, Django, React, PostgreSQL, Docker
   - Team: 20-person tech team
   - Perks: Hybrid work, conference budget
   - Match Score: 75% (requires some frontend)

**Application Strategy:**
- Priority: TechCorp Paris, FinanceAI (highest match)
- Quick wins: DataFlow Systems (skill match despite lower salary)
- Stretch: GreenTech Solutions (upskill on serverless)
"""
    return jobs


@tool
def generate_cover_letter(job_description: str, cv_summary: str, company_name: str) -> str:
    """Generate a tailored cover letter for a specific job application.

    Args:
        job_description: The job posting text
        cv_summary: Summary of the candidate's experience
        company_name: Name of the target company

    Returns:
        Customized cover letter text
    """
    letter = f"""Subject: Application for Backend Python Developer Position

Dear Hiring Manager at {company_name},

I am writing to express my strong interest in the Backend Python Developer position at {company_name}. With three years of hands-on Python development experience, I am confident in my ability to contribute effectively to your engineering team.

Throughout my career, I have developed robust backend systems using Python, demonstrating proficiency in building scalable applications. My experience includes:

- Designing and implementing RESTful APIs that serve thousands of daily requests
- Optimizing database queries resulting in significant performance improvements
- Collaborating with cross-functional teams to deliver features on schedule

I am particularly drawn to {company_name} because of your focus on modern technology stacks and commitment to engineering excellence. I am currently expanding my expertise in FastAPI, Docker, and AWS to align with industry trends and your technical requirements.

I am eager to bring my technical skills, problem-solving abilities, and enthusiasm for continuous learning to your team. I would welcome the opportunity to discuss how my background aligns with your needs.

Thank you for considering my application. I look forward to speaking with you soon.

Best regards,
[Your Name]
"""
    return letter


@tool
def research_salary_negotiation(role: str, location: str, years_experience: int) -> str:
    """Research salary bands and negotiation strategies for a specific role.

    Args:
        role: Job title
        location: Geographic location
        years_experience: Years of relevant experience

    Returns:
        Salary research and negotiation guidance
    """
    guidance = f"""Salary Negotiation Guide for {role} in {location} ({years_experience} years exp):

**Market Data:**
- 25th percentile: €48,000
- Median (50th): €55,000
- 75th percentile: €63,000
- Your target (€55,000): At market median ✓

**Your Negotiation Position:**
- Target is realistic and data-backed
- Room to negotiate up to €58,000-€60,000 with the right company
- Should accept no less than €52,000 given your experience

**Negotiation Strategy:**

1. **Initial Discussion:**
   - If asked about expectations: "€55,000-€60,000 based on my research for this role in Paris"
   - Justify with: market data + your 3 years experience + specific skills

2. **Key Talking Points:**
   - "I've researched the market rate for Python developers with 3 years experience in Paris"
   - "My backend development expertise and proven track record justify this range"
   - "I'm open to discussion based on the full compensation package"

3. **If Offered Lower (e.g., €50,000):**
   - "I appreciate the offer. Based on my research and experience, I was expecting closer to €55,000."
   - "Can we explore options to reach that number, either through base salary or other benefits?"

4. **Beyond Salary:**
   - Remote work flexibility
   - Professional development budget
   - Performance review timeline (6-month check-in)
   - Additional vacation days
   - Stock options or bonus structure

5. **Red Flags:**
   - Salary below €48,000 (significantly below market)
   - Vague promises of "future raises"
   - Pressure to accept immediately without time to consider

**Final Tips:**
- Always negotiate - companies expect it
- Get everything in writing
- Don't accept the first offer immediately
- Be prepared to walk away if significantly below target
"""
    return guidance

## Agent Construction

### System Prompt Design

The system prompt defines the agent's:
- **Role**: Career coaching expert
- **Capabilities**: Available tools and when to use them
- **Behavior**: Multi-step reasoning, thoroughness, structured output

A well-designed system prompt is critical for reliable agent behavior. It should:
- Explicitly list all available tools
- Provide examples of when to use each tool
- Define the expected workflow
- Specify output format

### Model Selection

We use GPT-4 for its:
- Superior reasoning capabilities
- Reliable tool selection
- Ability to follow complex instructions

Temperature is set to 0 for deterministic, focused reasoning.

In [7]:
# Initialize the LLM with tools
tools = [
    analyze_cv,
    research_market_trends,
    find_learning_resources,
    search_jobs,
    generate_cover_letter,
    research_salary_negotiation
]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# System prompt that defines agent behavior
system_prompt = """You are a professional career coaching agent. Your goal is to provide comprehensive career guidance.

Available tools:
1. analyze_cv: Identify CV weaknesses and improvement areas
2. research_market_trends: Research job market, in-demand skills, and salaries
3. find_learning_resources: Find courses and resources to fill skill gaps
4. search_jobs: Search for relevant job opportunities
5. generate_cover_letter: Create tailored cover letters for applications
6. research_salary_negotiation: Provide salary negotiation strategies

When a user provides their career goals and CV, you should:
1. Analyze their CV to identify weaknesses
2. Research current market trends for their target role
3. Find learning resources to address skill gaps
4. Search for relevant job opportunities
5. Generate sample cover letters for top matches
6. Provide salary negotiation guidance

Be thorough, use multiple tools, and provide actionable advice.
"""

def create_agent_node(state: AgentState) -> AgentState:
    """Agent node: LLM decides next action (tool call or final response)."""
    messages = state["messages"]

    # Prepend system prompt to conversation
    messages_with_system = [HumanMessage(content=system_prompt)] + list(messages)

    # LLM generates response (may include tool calls)
    response = llm_with_tools.invoke(messages_with_system)

    return {"messages": [response]}


def should_continue(state: AgentState) -> str:
    """Conditional edge: determine if agent should continue with tools or finish."""
    messages = state["messages"]
    last_message = messages[-1]

    # If the last message has tool calls, route to tools node
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"

    # Otherwise, finish
    return "end"

## Graph Construction

The graph defines the agent's execution flow:

```
[User Input] → [Agent Node] → [Decision]
                     ↑              |
                     |              ↓
                [Tool Node] ← [Continue?]
                                   |
                                   ↓
                                 [END]
```

**Flow:**
1. User provides input
2. Agent Node: LLM reasons and decides which tool(s) to call
3. Decision: If tool calls exist, route to Tool Node; otherwise, finish
4. Tool Node: Execute tool(s) and return results
5. Loop back to Agent Node with tool results
6. Repeat until agent decides to finish

This architecture enables:
- **Multi-step reasoning**: Agent can chain multiple tool calls
- **Dynamic planning**: Agent adjusts strategy based on tool outputs
- **Robustness**: Agent can recover from tool errors

In [8]:
# Build the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", create_agent_node)
workflow.add_node("tools", ToolNode(tools))

# Set entry point
workflow.set_entry_point("agent")

# Add conditional edges
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)

# Tool node always returns to agent
workflow.add_edge("tools", "agent")

# Compile the graph
app = workflow.compile()

print("✓ Career Coach Agent initialized successfully")
print("\nGraph structure:")
print("  1. agent → decision")
print("  2. decision → tools (if tool calls) or END")
print("  3. tools → agent (loop)")

✓ Career Coach Agent initialized successfully

Graph structure:
  1. agent → decision
  2. decision → tools (if tool calls) or END
  3. tools → agent (loop)


## Alternative: Using HuggingFace Models with Tool Calling

### Motivation for Open-Source Models

While GPT-4 provides excellent tool-calling capabilities, open-source models from HuggingFace offer several advantages:

**Benefits:**
- **Cost**: Free inference (local) or low-cost API endpoints
- **Privacy**: Run locally without sending data to third parties
- **Customization**: Fine-tune models for domain-specific tasks
- **Control**: No API rate limits or service dependencies

**Tradeoffs:**
- **Performance**: Generally lower tool-calling accuracy than GPT-4
- **Resource requirements**: Require significant GPU memory for local inference
- **Integration complexity**: May need custom parsing for tool calls

### Models with Native Tool Calling Support

Several HuggingFace models support structured tool calling:

1. **meta-llama/Meta-Llama-3.1-8B-Instruct** (8B parameters)
   - Native function calling support
   - Moderate hardware requirements (16GB+ VRAM)
   - Good balance of performance and efficiency

2. **NousResearch/Hermes-2-Pro-Llama-3-8B** (8B parameters)
   - Specifically trained for function calling
   - Excellent tool selection accuracy
   - Widely used in production agentic systems

3. **meetkai/functionary-small-v2.5** (7B parameters)
   - Optimized exclusively for tool calling
   - Lower resource requirements
   - Fast inference speed

4. **meta-llama/Meta-Llama-3.1-70B-Instruct** (70B parameters)
   - Superior reasoning capabilities
   - Requires significant GPU resources (A100 80GB or multi-GPU)
   - Closest open-source alternative to GPT-4 for tool calling

### Recommended Choice: Llama 3.1 8B Instruct

For this demonstration, we recommend **Llama 3.1 8B Instruct** because:
- Officially supports tool calling in the same format as OpenAI
- Well-documented and widely tested
- Reasonable hardware requirements
- Strong performance on multi-step reasoning tasks

**Hardware Requirements:**
- **Local inference**: 16GB+ GPU VRAM (RTX 4090, A100, etc.)
- **Cloud inference**: Free tier available on HuggingFace Inference API
- **Quantized versions**: 4-bit quantization reduces to ~6GB VRAM

## Example Execution: Complete Career Analysis

### User Input

The following prompt contains:
- **Current situation**: Python developer, 3 years experience, Paris
- **Target salary**: €55,000
- **CV**: Simulated CV text (in practice, user would paste actual CV)

### Expected Agent Behavior

The agent should:
1. Call `analyze_cv` to identify weaknesses
2. Call `research_market_trends` to understand market demands
3. Call `find_learning_resources` based on identified gaps
4. Call `search_jobs` to find opportunities
5. Call `generate_cover_letter` for top matches
6. Call `research_salary_negotiation` for guidance
7. Synthesize all results into actionable recommendations

### Observation of Agent Actions

During execution, we'll see:
- **Tool calls**: Which tools the agent selects and why
- **Tool outputs**: Results returned by each tool
- **Agent reasoning**: How the agent uses tool outputs to inform next steps

In [13]:
# Example user prompt
user_prompt = """I'm a Python developer with 3 years experience in Paris.
My target salary is €55k.

Here's my CV:

---
John Doe
Python Developer
Paris, France

EXPERIENCE:
Software Developer at StartupXYZ (2021-2024)
- Developed backend services using Python
- Worked with REST APIs
- Collaborated with team of 5 developers

Junior Developer at TechCompany (2020-2021)
- Built web applications
- Fixed bugs and maintained code

EDUCATION:
Bachelor in Computer Science - University of Paris (2020)

SKILLS:
Python, Git, SQL, Linux
---

Find opportunities and tell me how to improve my chances.
"""

print("=" * 80)
print("CAREER COACH AGENT EXECUTION")
print("=" * 80)
print("\n📝 User Request:")
print(user_prompt)
print("\n" + "=" * 80)

CAREER COACH AGENT EXECUTION

📝 User Request:
I'm a Python developer with 3 years experience in Paris.
My target salary is €55k.

Here's my CV:

---
John Doe
Python Developer
Paris, France

EXPERIENCE:
Software Developer at StartupXYZ (2021-2024)
- Developed backend services using Python
- Worked with REST APIs
- Collaborated with team of 5 developers

Junior Developer at TechCompany (2020-2021)
- Built web applications
- Fixed bugs and maintained code

EDUCATION:
Bachelor in Computer Science - University of Paris (2020)

SKILLS:
Python, Git, SQL, Linux
---

Find opportunities and tell me how to improve my chances.




In [14]:
# Execute the agent and track actions
config = {"recursion_limit": 50}
inputs = {"messages": [HumanMessage(content=user_prompt)]}

print("\n🤖 AGENT ACTIONS:\n")

step_count = 0
for output in app.stream(inputs, config=config):
    for key, value in output.items():
        if key == "agent":
            step_count += 1
            last_message = value["messages"][-1]

            # Check if agent is calling tools
            if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                print(f"\n{'─' * 80}")
                print(f"Step {step_count}: AGENT DECISION - Calling tools")
                print(f"{'─' * 80}")

                for tool_call in last_message.tool_calls:
                    tool_name = tool_call['name']
                    tool_args = tool_call['args']
                    print(f"\n🔧 Tool: {tool_name}")
                    print(f"   Arguments: {tool_args}")
            else:
                # Agent is providing final response
                print(f"\n{'─' * 80}")
                print(f"Step {step_count}: AGENT DECISION - Finishing")
                print(f"{'─' * 80}")

        elif key == "tools":
            print(f"\n✓ Tool execution completed")
            for msg in value["messages"]:
                if isinstance(msg, ToolMessage):
                    print(f"   → Result length: {len(msg.content)} characters")

print(f"\n\n{'=' * 80}")
print("FINAL AGENT RESPONSE")
print("=" * 80)


🤖 AGENT ACTIONS:


────────────────────────────────────────────────────────────────────────────────
Step 1: AGENT DECISION - Calling tools
────────────────────────────────────────────────────────────────────────────────

🔧 Tool: analyze_cv
   Arguments: {'cv_text': 'John Doe\nPython Developer\nParis, France\n\nEXPERIENCE:\nSoftware Developer at StartupXYZ (2021-2024)\n- Developed backend services using Python\n- Worked with REST APIs\n- Collaborated with team of 5 developers\n\nJunior Developer at TechCompany (2020-2021)\n- Built web applications\n- Fixed bugs and maintained code\n\nEDUCATION:\nBachelor in Computer Science - University of Paris (2020)\n\nSKILLS:\nPython, Git, SQL, Linux', 'target_role': 'Python Developer'}

🔧 Tool: research_market_trends
   Arguments: {'role': 'Python Developer', 'location': 'Paris'}

🔧 Tool: search_jobs
   Arguments: {'role': 'Python Developer', 'location': 'Paris', 'salary_min': 55000}

🔧 Tool: research_salary_negotiation
   Arguments: {'role': 'Pyt

In [15]:
# Display the final response
final_state = app.invoke(inputs, config=config)
final_message = final_state["messages"][-1]

print("\n" + final_message.content)
print("\n" + "=" * 80)


Here's a comprehensive plan to improve your career prospects as a Python Developer in Paris:

### CV Analysis
**Strengths:**
- 3 years of Python development experience
- Experience with backend development

**Weaknesses Identified:**
1. Missing modern Python frameworks (FastAPI, Django)
2. No mention of cloud platforms (AWS, Azure, GCP)
3. Limited testing/CI-CD experience
4. Absence of containerization skills (Docker, Kubernetes)
5. No quantifiable achievements or impact metrics

**Recommendations:**
- Add metrics to demonstrate impact (e.g., "Optimized API reducing response time by 40%")
- Highlight any cloud or DevOps experience
- Include collaborative projects or team leadership
- Mention any open-source contributions

### Market Trends
- **Demand:** High (15% YoY growth in job postings)
- **Competition:** Moderate (avg 45 applications per posting)
- **Average Salary:** €45,000 - €65,000 (mid-level)

**Top In-Demand Skills:**
1. FastAPI / Django / Flask
2. Docker & Kubernetes
3. AW

## Analysis of Agent Behavior

### Multi-Step Reasoning

The agent demonstrates sophisticated reasoning by:
1. **Sequential tool use**: Calls tools in logical order (CV analysis → market research → learning resources → job search)
2. **Context awareness**: Uses outputs from earlier tools to inform later tool calls
3. **Synthesis**: Combines information from multiple tools into coherent recommendations

### Tool Selection Strategy

The agent selects tools based on:
- **User goals**: Identified from the prompt (find jobs, improve chances)
- **Available information**: What data is needed to fulfill the goals
- **Tool descriptions**: Matches task requirements to tool capabilities

### LangGraph Advantages

This architecture provides:
- **Modularity**: Tools can be added/removed without changing graph structure
- **Observability**: Each step is traceable (agent decision → tool execution → result)
- **Error handling**: Graph can route around failed tools or retry
- **State persistence**: Conversation history maintained across tool calls

### Comparison to Simple Prompting

| Aspect | Simple Prompt | LangGraph Agent |
|--------|--------------|------------------|
| Tool access | Manual function calling | Automatic tool selection |
| Multi-step tasks | Single LLM call | Iterative execution loop |
| Error recovery | No built-in mechanism | Graph can handle failures |
| Observability | Limited | Full execution trace |
| Scalability | Hard to extend | Modular tool addition |

## Extending the Agent

### Adding New Tools

To extend functionality, simply:
1. Define a new function with `@tool` decorator
2. Add it to the `tools` list
3. Update system prompt to describe the new tool

Example new tools:
- **LinkedIn profile optimizer**: Analyze and improve LinkedIn profiles
- **Interview question generator**: Create practice questions for target roles
- **Company culture researcher**: Investigate company values and work environment
- **Network connection finder**: Identify mutual connections at target companies

### Production Considerations

For real-world deployment:
1. **Tool implementation**: Replace simulated tools with actual API calls (job boards, course platforms)
2. **Error handling**: Add try-except blocks in tools for API failures
3. **Rate limiting**: Implement delays between tool calls to respect API limits
4. **Caching**: Cache tool results to reduce API costs
5. **Cost monitoring**: Track token usage for each agent execution
6. **User privacy**: Ensure CV data is not stored or logged
7. **Result validation**: Verify tool outputs before presenting to users

### Advanced Features

Potential enhancements:
- **Human-in-the-loop**: Allow user to approve/modify agent decisions
- **Memory**: Store user preferences and past interactions
- **Multi-agent**: Separate specialized agents (CV expert, job search specialist, negotiation coach)
- **Streaming**: Stream agent thoughts and tool outputs in real-time
- **Evaluation**: Automated testing of agent performance on career coaching tasks

## Summary

This notebook demonstrated:

1. **LangGraph architecture**: State management, nodes, edges, and execution flow
2. **Tool integration**: How LLMs select and invoke external functions
3. **Multi-step reasoning**: Agent chains multiple tool calls to solve complex tasks
4. **Practical application**: Career coaching agent with 6 specialized tools
5. **Observability**: Tracking agent decisions and tool executions

### Key Takeaways

- **Agentic systems** combine LLM reasoning with tool-calling capabilities
- **LangGraph** provides a structured framework for building reliable agents
- **Tool design** is critical: clear names, descriptions, and function signatures
- **System prompts** guide agent behavior and tool selection strategy
- **Graph structure** enables complex workflows with conditional logic

### Further Reading

- LangGraph documentation: https://langchain-ai.github.io/langgraph/
- ReAct pattern (Reason + Act): https://arxiv.org/abs/2210.03629
- Tool use in LLMs: https://arxiv.org/abs/2302.04761